# Testing the model

we load the finetuned model. After that we load the test pictures from the path which is given in the test_data.csv . We've predicted on the test images and updated the database with the predicted genders and ages and saved into a new csv file. 

In [6]:
# Import all packages
from keras.models import load_model

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import sys
from keras.preprocessing.image import img_to_array
import tensorflow as tf
import pickle
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [7]:
project_path = "/home/mate/develop/PycharmProjects/GeFace/"

# load the trained convolutional neural network from disk, followed
# by the category and color label binarizers, respectively
print("[INFO] loading network...")
model = load_model(project_path  + "faces_model/last/xception_regression_age_fine.hdf5", custom_objects={"tf": tf})
agesLB = pickle.loads(open(project_path + "faces_model/last/regression.label", "rb").read())


[INFO] loading network...


/home/mate/.virtualenvs/deeplearn_gpu/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
# load the test database
test_csv = "/home/mate/develop/PycharmProjects/GeFace/test/test_data.csv"
df = pd.read_csv(test_csv)

In [9]:
df.head()

,Unnamed: 0,x_test_path,y_test_age,y_test_gender
0,0,faces_colored/73/nm0000673_rm1920911360_1964-1...,23.0,f
1,1,faces_colored/37/nm0005137_rm1375568640_1961-4...,49.0,f
2,2,faces_colored/00/nm0321800_rm1232569088_1982-1...,28.0,f
3,3,faces_colored/73/nm0416673_rm1596496128_1965-4...,42.0,m
4,4,faces_colored/68/nm0001368_rm13947136_1945-12-...,52.0,m


In [ ]:
# classify the input image using Keras' multi-output functionality
# do the prediction Batch by batch. Batch size is 128, we have about 26e3 test images
BS = 128
df['preds_age'] = 0
df['preds_gender'] = -1
for j in range(len(df)//BS):
    b_images = []
    for i in range(BS):
        image = cv2.imread(project_path+df['x_test_path'][j*BS+i])
        image = img_to_array(image)
        b_images.append(image)
    b_images = np.asarray(b_images)
    b_images = b_images.astype("float") / 255.0*2.0-1
    (agesProba) = model.predict(b_images)
    agesLabels = agesLB.inverse_transform(agesProba[0])
    genderLabels = agesProba[1]
    for ii in range(BS):
        df['preds_age'][j*BS+ii] = agesLabels[ii]
        df['preds_gender'][j*BS+ii] = np.argmax(genderLabels[ii])

In [ ]:
# saving into a new csv file
df.to_csv("/home/mate/develop/PycharmProjects/GeFace/test/test_result.csv")